In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
from tqdm import tqdm 

In [ ]:
# 0.장치(device) 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'# cpu도 한번해보기(이정도 data는 cpu가능하지만 조금만 데이터많아져도 gpu로 해야함)
              # 여기서 알아햘것: 이코드하나만 바꾸면 다른건 안바꿔도 된다느것!
print(f'Using device: {device}')



Using device: cpu


In [12]:
# 1. 데이터 준비
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

training_set = torchvision.datasets.FashionMNIST( #Dataset 생성
    './data',
    train=True,
    download=True,
    transform = transform
)
training_dataloader = DataLoader(
    training_set,batch_size=64, shuffle=True, # eval(평가) dataset은 셔플 하면 X
)


In [13]:
#2.1 모델 생성
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # ex)2차원으로 들어온넘 -> 1차원으로 세우기

        self.layers = nn.Sequential(
            nn.Linear(28*28,512), # 한장씩 내적x, 한방에 다 내적됨
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,10) # 10 이 가장중요: 10진분류이기에 마지막 나오는게 10이 되야함
        )                     # <실용> 대화용 AI가 까먹는 이유 --> 마지막층이 제한되있어서
    def forward(self,x):
        x = self.flatten(x) 
        # (64,1,28,28) -> (64,784) : 4차원이 2차원으로 준다.
        # 원래 차원이   
        return self.layers(x)



In [14]:
# 다만든 신경망을 GPU로 보내야함
model = MLP().to(device) # model을 cuda로 보내겠다.

In [15]:
# 3. 다중(10) 분류 
loss_fn = nn.CrossEntropyLoss() # 다중분류의 손실함수
optimizer = optim.Adam(model.parameters(),lr=0.001) # 설정한 lr 이정답이 아닐수 있다.

In [16]:
# 훈련 loop 돌리기
num_epochs = 5
for epoch in range(num_epochs): # 외부for문: epoch
                                # 5번 반복
    running_loss = 0.0

    # 지금부터 중요!! 모델을 훈련 모드로 설정.Dropout, BatchNorm
    model.train()  # <주의>훈련모드를 설정하는거임, 훈련하는건 아님
                   # <선언>지금부터 훈련할거야~~~를 알리는 거임
    progress_bar = tqdm(training_dataloader,desc=f'Epoch {epoch+1}/{num_epochs}') # 문자열도 f-스트링 사용많이함

    for i,dataset in enumerate(progress_bar): # <중요>진행되는 바 보여줌 # 내부for문:dataLoader순회
                                           # for ~ in : 내부적으로 iter next 임-> 다음을 반복해라
        # 1. 데이터 로드 및 cuda로 이동
        inputs, labels = dataset
        #<중요> 데이터를 메모리에 넣어뒀다가 학습에 필요할때만 cuda에 넣겠다!!!
        inputs, labels = inputs.to(device),labels.to(device) # data,신경망 둘다 gpu에 들어가 있음

        # 2.기울기 초기화
        optimizer.zero_grad()

        # 3.순전파 진행 -> 결과값(예측값)
        outputs = model(inputs)
        
        # 4.오차 계산
        loss = loss_fn(outputs, labels) # 결과값,원래답
                                        # loss는 tensor임 output,labels도 tensor여서
        
        # 5.역전파 : 기울기(미분) 구한다.
        loss.backward()

        # 6. 파라미터(w) 업데이트
        optimizer.step()
        # 통계 및 진행 바 업데이트
        running_loss += loss.item() # 누적
                                    # 오차에서 값을 뽑아낼땐 item으로 뽑아내야함
        progress_bar.set_postfix({'loss':running_loss/(i+1)})

    # 1이 에폭이 끝났을때
    avg_epoch_loss = running_loss / len(training_dataloader)
    print(f'End of Epoch {epoch+1}, Average Loss: {avg_epoch_loss:.4f}')

# 모든 에폭이 종료 --> 학습종료
print('\n Finished Traning!')

Epoch 1/5: 100%|██████████| 938/938 [00:18<00:00, 49.63it/s, loss=0.483]


End of Epoch 1, Average Loss: 0.4834


Epoch 2/5: 100%|██████████| 938/938 [00:18<00:00, 50.02it/s, loss=0.361]


End of Epoch 2, Average Loss: 0.3609


Epoch 3/5: 100%|██████████| 938/938 [00:19<00:00, 49.27it/s, loss=0.326]


End of Epoch 3, Average Loss: 0.3259


Epoch 4/5: 100%|██████████| 938/938 [00:19<00:00, 47.38it/s, loss=0.301]


End of Epoch 4, Average Loss: 0.3008


Epoch 5/5: 100%|██████████| 938/938 [00:22<00:00, 42.24it/s, loss=0.282]

End of Epoch 5, Average Loss: 0.2818

 Finished Traning!
